# Normal in vitro clock

The circadian clock of cyanobacteria *Synechococcus elongatus PCC 7942* can be reconstructed *in vitro*.
This protocol samples such an *in vitro* clock with uniform time intervals.

2024/8/23 Remove dye pipetting commands. Will need to add dye before the robot run

2025/1/10 Decreased sample volume from 6ul to 2ul. There isn't much of a point in keeping a "backup volume".
Change sampling interval to 3.5 hours so that the sampling scheme is not in sync with the circadian program

2025/2/15 Increased safe volume from 20 to 30 ul and lowered the aspiration height from 2 to 1.5 mm to avoid missing the last few samples

## Initialize the protocol

In [1]:
# current working dir is /var/lib/jupyter/notebooks
import opentrons.execute
from opentrons import protocol_api
import numpy as np
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous, calibrated_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *
import time

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.22")

# home is required
protocol.set_rail_lights(False)
protocol.home()

## Load labware

- The thermocycler is mounted to slot 7 and 10 (can't be changed)
- The Bio-rad/Eppendorf PCR plate sits in the thermocycler (to hold both the reaction and the samples)
- The GEB tip racks are on slot 4 and 5 (closest slots to the thermocycler to minimize moving time)
- The P10 1st gen pipette is on the left

In [2]:
tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '4')

tc_mod = protocol.load_module(module_name='thermocyclerModuleV2')

# Use Eppendorf twintec LoBind plates cat:14-380-052
# it has similar enough dimensions to the Biorad one
plate = tc_mod.load_labware(name='biorad_96_wellplate_200ul_calibrated')

pipette = protocol.load_instrument("p10_single", "left", 
                                   tip_racks = [tip_rack_1])

## Sampling and liquid settings


In [3]:
#!!! change to 4 hours in an experimental run !!!
dt_sample = 4*60    # time interval in minutes
# dt_sample = 5

t_relax = 24*60    # minutes
# t_relax = 10

v_sample = 2    # ul

SAMPLE_OFFSET = 32    # define the starting well: only use the right half of the plate. First sample well is A7
SAMPLE_OFFSET_2 = 64

RXN_TUBE = plate.wells_by_name()["C2"]
RXN_TUBE_2 = plate.wells_by_name()["C3"]


### Reaction volume

V(reaction tube) = 30 ul (margin volume) + 25*2 ul = 80 ul

To make the reaction mixture, mix 1.5 uM of KaiA, 3.5 uM of KaiB, and 3.5 uM of KaiC in Kai Storage Buffer (refer to Rust Lab Dropbox for recipe. It is a Tris-HCl-based buffer at pH 8 with Mg++) with 1 mM of ATP

Load well A5 to A8 (25 wells in total) and A9 to A12 with 3 ul of SDS samping buffer and 7ul of water (12 ul in total. The volume will be smaller by the end of the run due to evaporation)

Wipe down the rubber seal with ethanol before the run!

<div class="alert alert-warning">
    <h2>STOP!</h2>
    <br />
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?<br />
    Remember to add the dye before the robot run!!
</div>

## Instructions

In [4]:
def sample(idx, v_s):
    "Sampling the in vitro clock and seal with oil"
    
    tc_mod.open_lid()

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the sample
    transfer_viscous(pipette, protocol, v_s, RXN_TUBE, cur_well, 
                     asp_height=1.5, 
                     disp_height=0.5,
                     delay=0, 
                     if_touch_tip=False, 
                     if_mix=True, 
                     if_blowout=True)
    
    # 2nd set of samples
    # current well
    cur_well_2 = plate.wells()[idx + SAMPLE_OFFSET_2]

    # pipette the sample
    transfer_viscous(pipette, protocol, v_s, RXN_TUBE_2, cur_well_2, 
                     asp_height=1.5, 
                     disp_height=0.5,
                     delay=0, 
                     if_touch_tip=False, 
                     if_mix=True, 
                     if_blowout=True)
    
    tc_mod.close_lid()

## Running

In [5]:
tc_mod.close_lid()
tc_mod.set_lid_temperature(temperature=37)
tc_mod.set_block_temperature(temperature=30)

In [6]:
# make the lists of instructions
scheduler.drop()

# sampling
N = 25
str0 = "Sample and mix with loading buffer"
# first 24 hours is incubation
scheduler.cat(time_vec = np.arange(N)*dt_sample + t_relax,    # minutes. sample for 4 days including time 0
             func_vec = [sample]*N,
             param_vec = [ (i, v_sample) for i in range(N) ],
             str_vec = [str0]*N,
             n_tip_vec = [2]*N)

# show
scheduler.report(unit="hours")

The protocol requires a total of 50 tip(s)

At 24.00 hours, Sample and mix with loading buffer, with params (0, 2)
At 28.00 hours, Sample and mix with loading buffer, with params (1, 2)
At 32.00 hours, Sample and mix with loading buffer, with params (2, 2)
At 36.00 hours, Sample and mix with loading buffer, with params (3, 2)
At 40.00 hours, Sample and mix with loading buffer, with params (4, 2)
At 44.00 hours, Sample and mix with loading buffer, with params (5, 2)
At 48.00 hours, Sample and mix with loading buffer, with params (6, 2)
At 52.00 hours, Sample and mix with loading buffer, with params (7, 2)
At 56.00 hours, Sample and mix with loading buffer, with params (8, 2)
At 60.00 hours, Sample and mix with loading buffer, with params (9, 2)
At 64.00 hours, Sample and mix with loading buffer, with params (10, 2)
At 68.00 hours, Sample and mix with loading buffer, with params (11, 2)
At 72.00 hours, Sample and mix with loading buffer, with params (12, 2)
At 76.00 hours, Sample and mix

In [ ]:
# running
log_fn = "log/20250708_ivo_no_growth_WT_and_R393C_full_timecourse.log"
scheduler.run(protocol, log_fn)

In [8]:
# finish up

tc_mod.open_lid()
tc_mod.deactivate_lid()
tc_mod.deactivate_block()

In [10]:
tc_mod.close_lid()

'closed'